In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('./mushrooms.csv')

In [3]:
df = df.drop(columns=['veil-type'])

In [4]:
cla = {'e': 0, 'p': 1}
bruises = {'t': 0, 'f': 1}
g_spacing = {'c': 0, 'w': 1, 'd': 2}
g_size = {'b': 0, 'n': 1}
s_shape = {'e': 0, 't': 1}
r_number = {'n': 0, 'o': 1, 't': 2}

for df_d in [df]:
    df_d['class'] = df_d['class'].map(cla)
    df_d['bruises'] = df_d['bruises'].map(bruises)
    df_d['gill-spacing'] = df_d['gill-spacing'].map(g_spacing)
    df_d['gill-size'] = df_d['gill-size'].map(g_size)
    df_d['stalk-shape'] = df_d['stalk-shape'].map(s_shape)
    df_d['ring-number'] = df_d['ring-number'].map(r_number)

In [5]:
df = pd.get_dummies(df)

In [6]:
df

,class,bruises,gill-spacing,gill-size,stalk-shape,ring-number,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_b,cap-color_c,cap-color_e,cap-color_g,cap-color_n,cap-color_p,cap-color_r,cap-color_u,cap-color_w,cap-color_y,odor_a,odor_c,odor_f,odor_l,odor_m,odor_n,odor_p,odor_s,odor_y,gill-attachment_a,gill-attachment_f,gill-color_b,gill-color_e,gill-color_g,...,stalk-color-below-ring_b,stalk-color-below-ring_c,stalk-color-below-ring_e,stalk-color-below-ring_g,stalk-color-below-ring_n,stalk-color-below-ring_o,stalk-color-below-ring_p,stalk-color-below-ring_w,stalk-color-below-ring_y,veil-color_n,veil-color_o,veil-color_w,veil-color_y,ring-type_e,ring-type_f,ring-type_l,ring-type_n,ring-type_p,spore-print-color_b,spore-print-color_h,spore-print-color_k,spore-print-color_n,spore-print-color_o,spore-print-color_r,spore-print-color_u,spore-print-color_w,spore-print-color_y,population_a,population_c,population_n,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
3,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,0,1,1,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
8120,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
8121,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
8122,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('class', axis=1), df['class'], test_size=0.2, random_state=42)

In [8]:
X_test = X_test.to_numpy()
X_train = X_train.to_numpy()
Y_test = Y_test.to_numpy()
Y_train = Y_train.to_numpy()

### Логистическая регрессия

In [9]:
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

class LogisticRegressor:
  def __init__(self):
    self.loss = []

  def count_loss(self, x, y):
    z = np.dot(x, self.weights)
    predicted_true = y * np.log(sigmoid(z))
    predicted_false = (1 - y) * np.log(1 - sigmoid(z))
    return -np.sum(predicted_true + predicted_false) / x.shape[0]

  def fit(self, x, y, nepoch = 1000, lr = 0.01, log_loss = False):
    self.weights = np.random.rand(x.shape[1])
    for _ in range(nepoch):
      predicted = sigmoid(np.dot(x, self.weights))
      self.weights -= lr * np.dot(x.T, predicted - y) / x.shape[0]
      if log_loss:
        self.loss.append(self.count_loss(x, y))

  def predict(self, x):
    prediction = np.dot(x, self.weights)
    return [1 if predict > 0.5 else 0 for predict in sigmoid(prediction)]

In [10]:
lgr = LogisticRegressor()
lgr.fit(X_train, Y_train)
predicted = lgr.predict(X_test)
accuracy_score(predicted, Y_test)

0.8566153846153847

In [11]:
lr = LogisticRegressionCV()
lr.fit(X_train, Y_train)
predicted = lr.predict(X_test)
accuracy_score(predicted, Y_test)

1.0

In [12]:
hyperparameters = [
(0.1, 500), 
(0.1, 1000), 
(0.1, 1500),
(0.01, 5000),
(0.01, 10000),
(0.001, 5000),
(0.001, 10000),
(0.001, 15000)
]

for pair in hyperparameters:
    lr, nepoch = pair
    lgr = LogisticRegressor()
    lgr.fit(X_train, Y_train, nepoch = nepoch, lr = lr)
    predicted_test = lgr.predict(X_test)
    predicted_train = lgr.predict(X_train)
    print(f'Learning rate = {lr}, nepoch = {nepoch}, train accuracy = {accuracy_score(Y_train, predicted_train)}, test accuracy = {accuracy_score(Y_test, predicted_test)}')

Learning rate = 0.1, nepoch = 500, train accuracy = 0.9758424372980459, test accuracy = 0.9735384615384616
Learning rate = 0.1, nepoch = 1000, train accuracy = 0.9810740113863672, test accuracy = 0.9809230769230769
Learning rate = 0.1, nepoch = 1500, train accuracy = 0.988767502692722, test accuracy = 0.992
Learning rate = 0.01, nepoch = 5000, train accuracy = 0.9779966148638252, test accuracy = 0.9747692307692307
Learning rate = 0.01, nepoch = 10000, train accuracy = 0.9847668872134174, test accuracy = 0.9858461538461538
Learning rate = 0.001, nepoch = 5000, train accuracy = 0.8215110016925681, test accuracy = 0.8123076923076923
Learning rate = 0.001, nepoch = 10000, train accuracy = 0.9078319741498692, test accuracy = 0.9015384615384615
Learning rate = 0.001, nepoch = 15000, train accuracy = 0.9338359747653485, test accuracy = 0.9187692307692308


### Дерево решений

In [13]:
class Node:
  def __init__(self, index = None, value = None, left = None, right = None, gain = 0, predicted = None):
    self.index = index
    self.value = value
    self.left = left
    self.right = right
    self.gain = gain
    self.predicted = predicted


class ClassifyingDecisionTree:
  def __init__(self, max_depth = 2, min_samples_split = 2):
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split

  def fit(self, x, y):
    data = np.concatenate((x, y[:, np.newaxis]), axis = 1)
    self.root = self.build_tree(data)

  def predict(self, x):
    predicted = [self.single_prediction(obs, self.root) for obs in x]
    return predicted

  def single_prediction(self, x, node):
    if node.predicted != None:
      return node.predicted
    if x[node.index] <= node.value:
      return self.single_prediction(x, node.left)
    else:
      return self.single_prediction(x, node.right)

  def gini(self, rows):
    unique = np.unique(rows)
    impurity = 1
    for cls in unique:
      prob = len(rows[rows == cls]) / float(len(rows))
      impurity -= prob ** 2
    return impurity

  def gain(self, current_uncertainty, left, right):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * self.gini(left) - (1 - p) * self.gini(right)

  def split(self, data, index, value):
    left = np.array([item for item in data if item[index] <= value])
    right = np.array([item for item in data if item[index] > value])
    return left, right

  def find_split(self, data, num_features):
    split = Node()
    current_uncertainty = self.gini(data[:, -1])
    for index in range(num_features):
      values = data[:, index]
      for value in np.unique(values):
        left, right = self.split(data, index, value)
        if len(left) == 0 or len(right) == 0:
          continue
        gain = self.gain(current_uncertainty, left[:, -1], right[:, -1])
        if gain > split.gain:
          split.index = index
          split.value = value
          split.left = left
          split.right = right
          split.gain = gain
    return split

  def build_tree(self, data, current_depth = 0):
    X, Y = data[:, :-1], data[:, -1]
    num_samples, num_features = np.shape(X)
    if num_samples >= self.min_samples_split and current_depth <= self.max_depth:
      split = self.find_split(data, num_features)
      if split.gain > 0:
        left = self.build_tree(split.left, current_depth + 1)
        right = self.build_tree(split.right, current_depth + 1)
        split.left = left
        split.right = right
        return split
    value = max(list(Y), key = list(Y).count)
    return Node(predicted = value)

In [14]:
depths = [1, 2, 3]
for depth in depths:
  crt = ClassifyingDecisionTree(max_depth = depth)
  crt.fit(X_train, Y_train)
  predicted = crt.predict(X_test)
  print("Depth = {}, accuracy = {}".format(depth, accuracy_score(predicted, Y_test)))

Depth = 1, accuracy = 0.944
Depth = 2, accuracy = 0.9809230769230769
Depth = 3, accuracy = 0.9913846153846154


In [15]:
for depth in depths:
  sk_crt = DecisionTreeClassifier(max_depth = depth)
  sk_crt.fit(X_train, Y_train)
  predicted = sk_crt.predict(X_test)
  print("Depth = {}, accuracy = {}".format(depth, accuracy_score(predicted, Y_test)))

Depth = 1, accuracy = 0.8781538461538462
Depth = 2, accuracy = 0.944
Depth = 3, accuracy = 0.9809230769230769


### Random forest

In [16]:
from collections import Counter

class RandomForest:
  def __init__(self, num_trees = 10, max_depth = 2, min_samples_split = 2):
    self.num_trees = num_trees
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split
    self.forest = []

  def get_sample(self, x, y):
    random_rows = np.random.choice(a = x.shape[0], size = x.shape[0])
    return x[random_rows], y[random_rows]

  def fit(self, x, y):
    for _ in range(self.num_trees):
      sample_x, sample_y = self.get_sample(x, y)
      tree = ClassifyingDecisionTree(self.max_depth, self.min_samples_split)
      tree.fit(sample_x, sample_y)
      self.forest.append(tree)

  def predict(self, x):
    prediction = []
    y = []
    for tree in self.forest:
      y.append(tree.predict(x))
    y = np.swapaxes(a = y, axis1 = 0, axis2 = 1)  
    for obs in y:
      counter = Counter(obs)
      prediction.append(counter.most_common(1)[0][0])
    return prediction

In [17]:
num_trees = [2, 5, 10]
for n in num_trees:
  forest = RandomForest(num_trees = n)
  forest.fit(X_train, Y_train)
  predicted = forest.predict(X_test)
  print("Number of trees = {}, accuracy = {}".format(n, accuracy_score(predicted, Y_test)))

Number of trees = 2, accuracy = 0.9809230769230769
Number of trees = 5, accuracy = 0.9809230769230769
Number of trees = 10, accuracy = 0.9809230769230769


In [18]:
for n in num_trees:
  forest = RandomForestClassifier(n_estimators=n)
  forest.fit(X_train, Y_train)
  predicted = forest.predict(X_test)
  print("Number of trees = {}, accuracy = {}".format(n, accuracy_score(predicted, Y_test)))

Number of trees = 2, accuracy = 0.9993846153846154
Number of trees = 5, accuracy = 1.0
Number of trees = 10, accuracy = 1.0
